In [ ]:
pip install gspread oauth2client numpy matplotlib optuna

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import auth
import gspread
from google.auth import default
from tqdm import tqdm
import warnings
import optuna

warnings.filterwarnings('ignore')
optuna.logging.set_verbosity(optuna.logging.WARNING)
plt.style.use('seaborn-v0_8-whitegrid')
print("Libraries imported and settings configured.")

try:
    auth.authenticate_user()
    creds, _ = default()
    gc = gspread.authorize(creds)
    print("Authentication successful.")
except Exception as e:
    print(f"Authentication failed. Please ensure you are in a Colab environment. Error: {e}")

# ==============================================================================
# DATA LOADING AND PARSING
# ==============================================================================
def load_and_parse_sheet(sheet_url):
    """
    Loads data from the specified Google Sheet and parses all required parameters
    and program details into a configuration dictionary.
    """
    print("\nAttempting to open Google Sheet...")
    try:
        sheet = gc.open_by_url(sheet_url)
        worksheet = sheet.worksheet('Basic Emissions Model')
        all_values = worksheet.get_all_values()
        print("Google Sheet data loaded successfully.")
    except Exception as e:
        print(f"Fatal: Could not open or read the Google Sheet. Error: {e}")
        return None

    def find_value(search_term, col_offset=1):
        for row in all_values:
            for j, cell in enumerate(row):
                if search_term.lower() in str(cell).lower():
                    try:
                        value_str = row[j + col_offset]
                        return float(str(value_str).replace('$', '').replace('%', '').replace(',', '').strip())
                    except (ValueError, IndexError):
                        continue
        return None

    def parse_program_table(start_keyword, end_keyword, column_map):
        program_data = []
        in_program_section = False
        for row in all_values:
            row_str = ' '.join(str(c) for c in row)
            if start_keyword.lower() in row_str.lower():
                in_program_section = True
                continue
            if end_keyword.lower() in row_str.lower() and in_program_section:
                break
            if in_program_section and 'Bylder' in str(row[1]):
                try:
                    item = {}
                    for key, (col_idx, type_func) in column_map.items():
                        raw_val = row[col_idx].replace('%', '').replace(',', '').strip() if isinstance(row[col_idx], str) else row[col_idx]
                        item[key] = type_func(raw_val) if raw_val else 0
                    program_data.append(item)
                except (ValueError, IndexError) as e:
                    print(f"Warning: Skipping a row in '{start_keyword}' due to parsing error: {e}")
        return program_data

    config = {
        'total_supply': find_value('Total XETA Supply') or 2.1e10,
        'launch_price': find_value('Launch Price') or 0.002381,
        'initial_liquidity': find_value('Initial Liquidity in LP') or 1e6,
        'xeta_in_lp': find_value('Xeta in LP Pool') or 2.1e8,
        'box_price': find_value('Box Price') or 1350,
        'xeta_per_box': find_value('XETA/Box Sold') or 100,
        'monthly_recurring': find_value('Monthly Recurring') or 150,
        'node_owners_start': int(find_value('Node Owners Start') or 100),
        'node_owners_end': int(find_value('Node Owners End') or 15000),
        'node_uptime': (find_value('Node Uptime') or 80) / 100,
        'total_emissions': find_value('Total XETA Emissions') or 9.45e9,
        'monthly_emissions_rate': (find_value('Monthly Emissions Rate') or 1) / 100,
        'total_sellers': int(find_value('Total Sellers') or 150),
        'protocol_buyback_percent': (find_value('Buyback %') or 5.0) / 100,
        'monthly_sellers_sell': (find_value('Monthly Sellers Sell') or 75) / 100,
        'monthly_owners_sell': (find_value('Monthly Owners Sell') or 50) / 100,
        'lp_fee': 0.003
    }

    client_map = {'tier': (1, str), 'boxes_sold': (2, float), 'token_rewards': (3, float), 'percent_sellers': (4, lambda x: float(x)/100)}
    config['client_program'] = parse_program_table('Suggested Seller Program', 'Total', client_map)

    unit_zero_map = {'tier': (1, str), 'boxes_sold': (2, float), 'multiplier': (3, float), 'token_rewards': (4, float),
                     'node_emissions_percent': (5, lambda x: float(x)/100), 'percent_sellers': (6, lambda x: float(x)/100)}
    config['unit_zero_program'] = parse_program_table('Recommended Seller Program', 'Total', unit_zero_map)

    if config['unit_zero_program']:
        config['unit_zero_multipliers'] = [{'tier': t['tier'], 'multiplier': t['multiplier']} for t in config['unit_zero_program']]
        config['unit_zero_seller_emission_share'] = config['unit_zero_program'][0].get('node_emissions_percent', 0.25)
    else:
        config['unit_zero_multipliers'] = []
        config['unit_zero_seller_emission_share'] = 0.25

    growth = []
    for i, row in enumerate(all_values):
        if 'Node Owners' in str(row[1]):
            try:
                growth = [float(str(v).replace(',', '')) for v in row[2:15]]
                break
            except (ValueError, IndexError):
                continue
    config['node_growth'] = growth if len(growth) == 13 else np.linspace(config['node_owners_start'], config['node_owners_end'], 13).tolist()

    print("\n--- Configuration Loaded ---")
    if not config['client_program'] or not config['unit_zero_program']:
        print("Fatal: Could not parse one or both seller programs. Please check sheet structure.")
        return None
    return config

# ==============================================================================
# CORE SIMULATION CLASSES
# ==============================================================================

class Agent:
    def __init__(self, agent_id, agent_type): self.id, self.type, self.tokens_held = agent_id, agent_type, 0

class Seller(Agent):
    def __init__(self, agent_id, tier_info, box_behavior='steady', token_behavior='consistent'):
        super().__init__(agent_id, 'seller')
        self.tier_info = tier_info
        self.box_behavior = box_behavior
        self.token_behavior = token_behavior
        self.total_boxes_sold = 0
        self.monthly_box_targets = self._calculate_monthly_targets()
        # Flag to handle one-time rewards for the Client Program
        self.claimed_rewards = {tier_info['boxes_sold']: False}
        self.base_sell_ratio = {'hodler': 0.1, 'consistent': 0.5, 'full_sell': 1.0}.get(self.token_behavior, 0.5)

    def _calculate_monthly_targets(self):
        total_boxes = self.tier_info['boxes_sold']
        if total_boxes <= 0: return np.zeros(12)
        if self.box_behavior == 'steady':
            targets = np.random.normal(total_boxes / 12, total_boxes / 12 * 0.2, 12)
        else: # 'volatile'
            targets = np.random.exponential(total_boxes / 12, 12)
        return np.maximum(0, targets) * (total_boxes / targets.sum()) if targets.sum() > 0 else np.zeros(12)

    def sell_boxes_this_month(self, month):
        boxes_sold = self.monthly_box_targets[month]
        self.total_boxes_sold += boxes_sold
        return boxes_sold

class NodeOperator(Agent):
    def __init__(self, agent_id, token_behavior='balanced', price_sensitivity=1.0):
        super().__init__(agent_id, 'node_operator')
        self.token_behavior = token_behavior
        self.price_sensitivity = price_sensitivity

    def receive_and_process_tokens(self, rewards, token_price, launch_price, sell_pressure_factor):
        if rewards <= 0 or token_price <= 0: return 0, 0
        price_factor = np.clip((token_price / launch_price) ** self.price_sensitivity, 0.5, 2.0)
        base_sell_ratio = {'hodler': 0.1, 'balanced': 0.5, 'full_sell': 1.0}.get(self.token_behavior, 0.5)
        effective_sell_ratio = np.clip(base_sell_ratio * sell_pressure_factor * price_factor, 0, 1)
        tokens_to_sell = rewards * effective_sell_ratio
        self.tokens_held += rewards - tokens_to_sell
        return tokens_to_sell, tokens_to_sell * token_price

class LiquidityPool:
    def __init__(self, token_reserve, usd_reserve, fee):
        self.token_reserve, self.usd_reserve, self.fee = float(token_reserve), float(usd_reserve), float(fee)
    def get_price(self): return self.usd_reserve / self.token_reserve if self.token_reserve > 0 else 0
    def sell_tokens(self, token_amount):
        if token_amount <= 0: return 0
        usd_out = self.usd_reserve - (self.token_reserve * self.usd_reserve) / (self.token_reserve + token_amount)
        fee_in_usd = usd_out * self.fee
        self.token_reserve += token_amount; self.usd_reserve -= usd_out; self.usd_reserve += fee_in_usd
        return usd_out * (1 - self.fee)
    def buy_tokens(self, usd_amount):
        if usd_amount <= 0: return 0
        tokens_out = self.token_reserve - (self.token_reserve * self.usd_reserve) / (self.usd_reserve + usd_amount)
        fee_in_tokens = tokens_out * self.fee
        self.usd_reserve += usd_amount; self.token_reserve -= tokens_out; self.token_reserve += fee_in_tokens
        return tokens_out * (1 - self.fee)

class Simulation:
    def __init__(self, config, program_type, custom_multipliers=None):
        self.config, self.program_type, self.custom_multipliers = config, program_type, custom_multipliers
        self.is_unit_zero = 'unit_zero' in self.program_type
        self.lp = LiquidityPool(config['xeta_in_lp'], config['initial_liquidity'], config['lp_fee'])
        self._initialize_agents()

    def _initialize_agents(self):
        program = self.config['client_program'] if not self.is_unit_zero else self.config['unit_zero_program']
        self.sellers = []
        for tier in program:
            num_sellers_in_tier = int(self.config['total_sellers'] * tier['percent_sellers'])
            for _ in range(num_sellers_in_tier):
                seller = Seller(len(self.sellers), tier,
                                box_behavior=np.random.choice(['steady', 'volatile']),
                                token_behavior=np.random.choice(['hodler', 'consistent', 'full_sell']))
                self.sellers.append(seller)

        self.node_operators = [NodeOperator(i, np.random.choice(['hodler', 'balanced', 'full_sell']), np.random.uniform(0.5, 1.5))
                               for i in range(self.config['node_owners_start'])]

    def market_shock_event(self, shock_chance=0.1, shock_strength=0.1):
        if np.random.rand() < shock_chance:
            shock_type = np.random.choice(['buy', 'sell'])
            shock_amount_tokens = self.lp.token_reserve * shock_strength * np.random.rand()
            if shock_type == 'sell': self.lp.sell_tokens(shock_amount_tokens)
            else: self.lp.buy_tokens(shock_amount_tokens * self.lp.get_price())

    def run(self):
        history = []
        for month in range(12):
            target_nodes = self.config['node_growth'][month+1]
            new_nodes = max(0, int(target_nodes - len(self.node_operators)))
            self.node_operators.extend([NodeOperator(len(self.node_operators) + i, np.random.choice(['hodler', 'balanced'])) for i in range(new_nodes)])
            active_nodes = int(len(self.node_operators) * self.config['node_uptime'])
            if active_nodes == 0: continue

            total_boxes_sold_this_month = sum(s.sell_boxes_this_month(month) for s in self.sellers)
            box_sales_revenue = total_boxes_sold_this_month * self.config['box_price']
            recurring_revenue = len(self.node_operators) * self.config['monthly_recurring']
            total_monthly_revenue = box_sales_revenue + recurring_revenue

            total_monthly_emissions = self.config['total_emissions'] * self.config['monthly_emissions_rate']
            emissions_per_seller = 0
            if self.is_unit_zero:
                seller_share = self.config['unit_zero_seller_emission_share']
                emissions_for_sellers = total_monthly_emissions * seller_share
                emissions_for_operators = total_monthly_emissions * (1 - seller_share)
                emissions_per_seller = emissions_for_sellers / len(self.sellers) if self.sellers else 0
                emissions_per_node = emissions_for_operators / active_nodes
            else:
                emissions_for_operators = total_monthly_emissions
                emissions_per_node = emissions_for_operators / active_nodes

            monthly_seller_sell_from_box = 0
            monthly_seller_sell_from_emissions = 0
            monthly_operator_sell_pressure = 0
            monthly_operator_sell_usd = 0

            for seller in self.sellers:
                monthly_box_rewards = 0
                seller_effective_sell_ratio = seller.base_sell_ratio * self.config['monthly_sellers_sell']
                boxes_this_month = seller.monthly_box_targets[month]

                if not self.is_unit_zero:
                    # Logic for Client Program (lump-sum, one-time reward)
                    if not seller.claimed_rewards.get(seller.tier_info['boxes_sold'], False) and seller.total_boxes_sold >= seller.tier_info['boxes_sold']:
                        monthly_box_rewards = seller.tier_info['token_rewards']
                        seller.claimed_rewards[seller.tier_info['boxes_sold']] = True
                else:
                    # Logic for Unit Zero Program (continuous per-box reward with multiplier)
                    multipliers = self.custom_multipliers if self.custom_multipliers is not None else self.config['unit_zero_multipliers']
                    multiplier_val = next((m['multiplier'] for m in multipliers if m['tier'] == seller.tier_info['tier']), 1.0)
                    monthly_box_rewards = boxes_this_month * self.config['xeta_per_box'] * multiplier_val

                tokens_sold_from_box = monthly_box_rewards * seller_effective_sell_ratio
                tokens_sold_from_emissions = emissions_per_seller * seller_effective_sell_ratio
                seller.tokens_held += (monthly_box_rewards + emissions_per_seller) - (tokens_sold_from_box + tokens_sold_from_emissions)
                monthly_seller_sell_from_box += tokens_sold_from_box
                monthly_seller_sell_from_emissions += tokens_sold_from_emissions

            total_seller_sell_pressure = monthly_seller_sell_from_box + monthly_seller_sell_from_emissions

            active_operators = np.random.choice(self.node_operators, active_nodes, replace=False)
            current_price = self.lp.get_price()
            for op in active_operators:
                tokens_sold, usd_value = op.receive_and_process_tokens(emissions_per_node, current_price, self.config['launch_price'], self.config['monthly_owners_sell'])
                monthly_operator_sell_pressure += tokens_sold
                monthly_operator_sell_usd += usd_value

            self.lp.sell_tokens(total_seller_sell_pressure + monthly_operator_sell_pressure)
            self.lp.buy_tokens(total_monthly_revenue * self.config['protocol_buyback_percent'])
            self.market_shock_event()

            history.append({
                'month': month, 'price': self.lp.get_price(), 'revenue': total_monthly_revenue,
                'seller_sell_total': total_seller_sell_pressure,
                'operator_sell_tokens': monthly_operator_sell_pressure,
                'lp_token_reserve': self.lp.token_reserve, 'lp_usd_reserve': self.lp.usd_reserve,
                'sell_pressure_seller_box': monthly_seller_sell_from_box,
                'sell_pressure_seller_emission': monthly_seller_sell_from_emissions,
                'sell_pressure_operator_emission': monthly_operator_sell_pressure
            })
        return pd.DataFrame(history)

# ==============================================================================
# MONTE CARLO & OPTIMIZATION FUNCTIONS
# ==============================================================================
def run_monte_carlo(n_sims, config, program_type, custom_multipliers=None, show_progress=True):
    all_results = []
    if 'optimized' in program_type: desc = 'Unit Zero Optimized'
    elif 'unit_zero' in program_type: desc = 'Unit Zero Base'
    else: desc = 'Client Program'
    iterator = range(n_sims)
    if show_progress: iterator = tqdm(iterator, desc=f"Simulating {desc}")
    for i in iterator:
        sim = Simulation(config, program_type, custom_multipliers)
        result_df = sim.run()
        result_df['sim_id'] = i
        result_df['program'] = desc
        all_results.append(result_df)
    return pd.concat(all_results) if all_results else pd.DataFrame()

def analyze_results(df):
    if df.empty: return {}
    final_prices = df[df['month'] == 11].groupby('sim_id')['price'].first()
    volatility = df.groupby('sim_id')['price'].std() / df.groupby('sim_id')['price'].mean()
    total_revenue = df.groupby('sim_id')['revenue'].sum()
    return {'final_prices': final_prices.tolist(), 'volatility': volatility.tolist(), 'total_revenue': total_revenue.tolist(),
        'avg_final_price': final_prices.mean(), 'avg_volatility': volatility.mean(), 'avg_total_revenue': total_revenue.mean(),
        'median_final_price': final_prices.median(), 'p5_final_price': final_prices.quantile(0.05)}

def objective(trial, config):
    multipliers = [{'tier': t['tier'], 'multiplier': trial.suggest_float(f"mult_{t['tier']}", 0.1, 5.0)} for t in config['unit_zero_program']]
    results_df = run_monte_carlo(10, config, 'unit_zero_optimized', multipliers, show_progress=False)
    metrics = analyze_results(results_df)
    p5_price = metrics.get('p5_final_price', 0); avg_volatility = metrics.get('avg_volatility', 1)
    return p5_price * (1 - avg_volatility)

def optimize_with_optuna(n_trials, config):
    print(f"\nStarting Optuna optimization with {n_trials} trials...")
    study = optuna.create_study(direction='maximize')
    study.optimize(lambda trial: objective(trial, config), n_trials=n_trials)
    print("Optimization complete."); print(f"Best trial score: {study.best_value:.6f}")
    best_multipliers = [{'tier': t['tier'], 'multiplier': study.best_params[f"mult_{t['tier']}"]} for t in config['unit_zero_program']]
    print("Best multipliers found:")
    for m in best_multipliers: print(f"  - {m['tier']}: {m['multiplier']:.3f}x")
    return best_multipliers

# ==============================================================================
# EXECUTION AND VISUALIZATION
# ==============================================================================

SHEET_URL = 'https://docs.google.com/spreadsheets/d/1PvHsSDPO01Ah9YBdA3f9VGX5nFu5qAXr5e4MNqO1lUo/edit?gid=764462073#gid=764462073'
NUM_SIMULATIONS = 2000
OPTIMIZATION_TRIALS = 30

CONFIG = load_and_parse_sheet(SHEET_URL)

if CONFIG:
    client_results_df = run_monte_carlo(NUM_SIMULATIONS, CONFIG, 'client')
    unit_zero_base_results_df = run_monte_carlo(NUM_SIMULATIONS, CONFIG, 'unit_zero_base')
    optimized_multipliers = optimize_with_optuna(OPTIMIZATION_TRIALS, CONFIG)
    unit_zero_optimized_results_df = run_monte_carlo(NUM_SIMULATIONS, CONFIG, 'unit_zero_optimized', optimized_multipliers)
    all_results_df = pd.concat([client_results_df, unit_zero_base_results_df, unit_zero_optimized_results_df])
    client_metrics = analyze_results(client_results_df)
    unit_zero_base_metrics = analyze_results(unit_zero_base_results_df)
    unit_zero_optimized_metrics = analyze_results(unit_zero_optimized_results_df)

    # ---  Visualization ---
    fig, axes = plt.subplots(3, 2, figsize=(20, 26))
    fig.suptitle(f'XETA Seller Program {NUM_SIMULATIONS} Step MC Sim', fontsize=24, y=1.0)
    palette = sns.color_palette("viridis", 3)

    # A) Price Path Fan Chart
    ax = axes[0, 0]
    sns.lineplot(data=all_results_df, x='month', y='price', hue='program', ax=ax, palette=palette, errorbar=('pi', 90))
    ax.axhline(CONFIG['launch_price'], color='red', linestyle='--', label=f"Launch Price (${CONFIG['launch_price']:.4f})")
    ax.set_title('A) Price Path Distribution (5th-95th Percentile)', fontsize=14)
    ax.set_xlabel('Month'); ax.set_ylabel('Token Price ($)')
    ax.legend(title='Program')

    # B) Avg. Monthly Selling Pressure by Source (Bar Chart)
    ax = axes[0, 1]
    plot_data_b = all_results_df[['program', 'sell_pressure_seller_box', 'sell_pressure_seller_emission', 'sell_pressure_operator_emission']]
    plot_data_b = plot_data_b.melt(id_vars='program', var_name='source', value_name='tokens_sold')
    plot_data_b['source'] = plot_data_b['source'].map({
        'sell_pressure_seller_box': 'Seller - Box Rewards',
        'sell_pressure_seller_emission': 'Seller - Emissions',
        'sell_pressure_operator_emission': 'Node - Emissions'
    })
    avg_monthly_sells = plot_data_b.groupby(['program', 'source'])['tokens_sold'].mean().reset_index()
    sns.barplot(data=avg_monthly_sells, x='program', y='tokens_sold', hue='source', ax=ax)
    ax.set_title('B) Avg. Monthly Selling Pressure by Source', fontsize=14)
    ax.set_xlabel(''); ax.set_ylabel('Avg. Monthly Tokens Sold')
    ax.tick_params(axis='x', rotation=10)
    ax.legend(title='Source')

    # C) Combined Monthly Selling Pressure by Agent Type (Line Graph)
    ax = axes[1, 0]
    plot_data_c = all_results_df[['program', 'month', 'seller_sell_total', 'operator_sell_tokens']]
    plot_data_c = plot_data_c.rename(columns={'seller_sell_total': 'Seller Network', 'operator_sell_tokens': 'Node Operators'})
    plot_data_c = plot_data_c.melt(id_vars=['program', 'month'], var_name='source', value_name='tokens_sold')
    sns.lineplot(data=plot_data_c, x='month', y='tokens_sold', hue='program', style='source', ax=ax, palette=palette, errorbar='sd')
    ax.set_title('C) Monthly Selling Pressure by Source and Program', fontsize=14)
    ax.set_xlabel('Month'); ax.set_ylabel('Avg. Monthly Tokens Sold')
    ax.legend(title='Program / Source')

    # D) Monthly Selling Pressure from Seller Network (Original Plot C)
    ax = axes[1, 1]
    sns.lineplot(data=all_results_df, x='month', y='seller_sell_total', hue='program', ax=ax, palette=palette, errorbar='sd')
    ax.set_title('D) Monthly Selling Pressure from Seller Network', fontsize=14)
    ax.set_xlabel('Month'); ax.set_ylabel('Avg. Monthly Tokens Sold')
    ax.legend(title='Program')

    # E) Optimized Multipliers
    ax = axes[2, 0]
    tiers = [t['tier'].replace('Bylder ', '').replace('Executive', 'Exec') for t in optimized_multipliers]
    multipliers = [t['multiplier'] for t in optimized_multipliers]
    sns.barplot(x=multipliers, y=tiers, ax=ax, color='skyblue', orient='h')
    ax.set_title('E) Optimized Multipliers for Unit Zero', fontsize=14)
    ax.set_xlabel('Optimized Multiplier Value (x)'); ax.set_ylabel('')

    # F) Summary Statistics
    ax = axes[2, 1]
    ax.axis('off')
    summary_text = "Avg. Performance Metrics:\n\n"
    scenarios = {"Client Program": client_metrics, "Unit Zero Base": unit_zero_base_metrics, "Unit Zero Optimized": unit_zero_optimized_metrics}
    for name, metrics in scenarios.items():
        if metrics:
            summary_text += f"--- {name} ---\n"
            summary_text += f"Median Final Price: ${metrics.get('median_final_price', 0):.5f}\n"
            summary_text += f"5th Pctl Price:     ${metrics.get('p5_final_price', 0):.5f} (Worst Case)\n"
            summary_text += f"Total Revenue:      ${metrics.get('avg_total_revenue', 0):,.0f}\n"
            summary_text += f"Price Volatility (CV): {metrics.get('avg_volatility', 0):.3f}\n\n"
    ax.text(0.0, 0.95, summary_text, family='monospace', fontsize=12, va='top')
    ax.set_title('F) Scenario Performance Summary', fontsize=14, y=1.0)

    plt.tight_layout(rect=[0, 0, 1, 0.98])
    plt.show()
else:
    print("\nExecution halted due to configuration loading failure.")